In [1]:
!python -V

Python 3.10.5


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [ ]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db") # Esta línea establece la URI de seguimiento de MLflow para usar una base de datos SQLite local
mlflow.set_experiment("nyc-taxi-experiment") # Esta línea establece el experimento de MLflow en el que se registrarán los resultados

2025/05/23 10:29:46 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/05/23 10:29:47 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location=('file:///d:/Universidad/Cursos/MLOps en '
 'Zoomcamp/02-experiment-tracking/mlruns/1'), creation_time=1748017795294, experiment_id='1', last_update_time=1748017795294, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [10]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [11]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [12]:
len(df_train), len(df_val)

(73908, 61921)

In [13]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [14]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [15]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [20]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715203341164

In [23]:
with mlflow.start_run(run_name="NYC_taxi_model_01") as run:
    mlflow.set_tag("developer", "Daniel") #Etiqueta personalizada para identificar al desarrollador

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [24]:
import xgboost as xgb

In [25]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [26]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [29]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [30]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

d:\Universidad\Cursos\MLOps en Zoomcamp\.venv\lib\site-packages\xgboost\callback.py:386: UserWarning: [11:13:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:11.60418                          
[1]	validation-rmse:11.05460                          
[2]	validation-rmse:10.56030                          
[3]	validation-rmse:10.11553                          
[4]	validation-rmse:9.71749                           
[5]	validation-rmse:9.36177                           
[6]	validation-rmse:9.04356                           
[7]	validation-rmse:8.75896                           
[8]	validation-rmse:8.50832                           
[9]	validation-rmse:8.28350                           
[10]	validation-rmse:8.08513                          
[11]	validation-rmse:7.90893                          
[12]	validation-rmse:7.75427                          
[13]	validation-rmse:7.61644                          
[14]	validation-rmse:7.49493                          
[15]	validation-rmse:7.38879                          
[16]	validation-rmse:7.29380                          
[17]	validation-rmse:7.20987                          
[18]	valid

d:\Universidad\Cursos\MLOps en Zoomcamp\.venv\lib\site-packages\xgboost\callback.py:386: UserWarning: [11:14:09] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[2]	validation-rmse:7.41685                                                    
[3]	validation-rmse:7.09897                                                    
[4]	validation-rmse:6.95157                                                    
[5]	validation-rmse:6.87524                                                    
[6]	validation-rmse:6.83750                                                    
[7]	validation-rmse:6.81685                                                    
[8]	validation-rmse:6.79879                                                    
[9]	validation-rmse:6.78861                                                    
[10]	validation-rmse:6.78665                                                   
[11]	validation-rmse:6.77684                                                   
[12]	validation-rmse:6.77462                                                   
[13]	validation-rmse:6.76982                                                   
[14]	validation-rmse:6.76714            

d:\Universidad\Cursos\MLOps en Zoomcamp\.venv\lib\site-packages\xgboost\callback.py:386: UserWarning: [11:14:15] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:9.88829                                                    
[1]	validation-rmse:8.48845                                                    
[2]	validation-rmse:7.67808                                                    
[3]	validation-rmse:7.22473                                                    
[4]	validation-rmse:6.97246                                                    
[5]	validation-rmse:6.81998                                                    
[6]	validation-rmse:6.73122                                                    
[7]	validation-rmse:6.67231                                                    
[8]	validation-rmse:6.63789                                                    
[9]	validation-rmse:6.61265                                                    
[10]	validation-rmse:6.59656                                                   
[11]	validation-rmse:6.58440                                                   
[12]	validation-rmse:6.57389            

d:\Universidad\Cursos\MLOps en Zoomcamp\.venv\lib\site-packages\xgboost\callback.py:386: UserWarning: [11:15:09] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:11.60708                                                   
[1]	validation-rmse:11.06135                                                   
[2]	validation-rmse:10.57073                                                   
[3]	validation-rmse:10.12920                                                   
[4]	validation-rmse:9.73316                                                    
[5]	validation-rmse:9.37664                                                    
[6]	validation-rmse:9.06216                                                    
[7]	validation-rmse:8.78252                                                    
[8]	validation-rmse:8.52792                                                    
[9]	validation-rmse:8.30510                                                    
[10]	validation-rmse:8.10248                                                   
[11]	validation-rmse:7.92577                                                   
[12]	validation-rmse:7.76902            

KeyboardInterrupt: 

In [31]:
mlflow.xgboost.autolog(disable=True)

In [33]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.3653173474528154,
        'max_depth': 28,
        'min_child_weight': 1.7718047276185382,
        'objective': 'reg:linear',
        'reg_alpha': 0.08450957998522358,
        'reg_lambda': 0.3501155694433485,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

d:\Universidad\Cursos\MLOps en Zoomcamp\.venv\lib\site-packages\xgboost\callback.py:386: UserWarning: [11:34:17] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:9.43913
[1]	validation-rmse:7.98914
[2]	validation-rmse:7.27712
[3]	validation-rmse:6.93349
[4]	validation-rmse:6.76072
[5]	validation-rmse:6.66790
[6]	validation-rmse:6.61755
[7]	validation-rmse:6.58478
[8]	validation-rmse:6.56405
[9]	validation-rmse:6.55230
[10]	validation-rmse:6.54458
[11]	validation-rmse:6.54114
[12]	validation-rmse:6.53537
[13]	validation-rmse:6.53333
[14]	validation-rmse:6.52817
[15]	validation-rmse:6.52316
[16]	validation-rmse:6.52043
[17]	validation-rmse:6.51675
[18]	validation-rmse:6.51400
[19]	validation-rmse:6.50978
[20]	validation-rmse:6.50705
[21]	validation-rmse:6.50415
[22]	validation-rmse:6.50099
[23]	validation-rmse:6.49778
[24]	validation-rmse:6.49365
[25]	validation-rmse:6.49203
[26]	validation-rmse:6.48947
[27]	validation-rmse:6.48669
[28]	validation-rmse:6.48465
[29]	validation-rmse:6.48160
[30]	validation-rmse:6.48000
[31]	validation-rmse:6.47777
[32]	validation-rmse:6.47577
[33]	validation-rmse:6.47312
[34]	validation-rmse:6.4

d:\Universidad\Cursos\MLOps en Zoomcamp\.venv\lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [11:35:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/05/23 11:35:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
